<h1> Assignment (Week 4) Capstone Project #SoBerlin </h1><br><br>
<h1> The story </h1>

<h3> Ever been to Berlin? </h3>
<br>
If yes - you might have heard someone saying "omg thats so Berlin" <br>
<br>
It actually somewhat became a meme. However - what does it even mean to be "so Berlin" <br>
Is there any other city in Germany which is "so (like) Berlin"? <br>
<br> The main question here is: If you are happy in a German city, and your main criteria for relocation would be,
<br> that you would like to continue your favorite way of nightlife and that you would still need to earn a living 
<br> and live at some place
<br>
<br>
Hence, I decided to use: 
<br> a) The average Income
<br> b) The average Rental Prices 
<br> c) A foursquare Profile of possible activities for the night 
<br> <br>
To find clusters within German cities exceeding 500.000 inhabitants
<br>

In [ ]:
# Loading dependencies
from bs4 import BeautifulSoup # Used for wiki-scraping

import numpy as np # library to handle data in a vectorized manner
import requests

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

<h3> Part I </h3>
<br> Extracting the name of all German Cities with more than 500.000 Inhabitants with their respective Size

In [ ]:
# Loading the names of "Groß und Mittelstädte"-Table from Wikipedia to find the 700 Cities with > 20.000 Inhabitants
wiki_url = 'https://de.wikipedia.org/wiki/Liste_der_Gro%C3%9F-_und_Mittelst%C3%A4dte_in_Deutschland'
wiki = requests.get(wiki_url).text
html_soup=BeautifulSoup(wiki,'html.parser')
sov_tables = html_soup.find_all('table', class_="wikitable sortable zebra")
df = pd.read_html(str(sov_tables[1]))
df = df[0]

In [ ]:
# Additional Cleaning: Remove the dots, to facilitate a change of data type then Change no of inhabitants to a float type
df['2018'] = df['2018'].str.replace('.','')
df['2018'] = pd.to_numeric(df['2018'], errors='coerce')

In [ ]:
# Remove Cities with less then 300.000 Inhabitants and columns which aren't needed
df=df[df['2018'] > 500000]
df.drop(['Rang','1970','1980','1990','2000','2010','Bundesland'],axis=1, inplace=True)

<h3> Done... Extracted all the Names of German Cities with more than 300 000 Inhabitants

In [ ]:
# Check new data frame
df.columns = ['Stadt','Size']
df

<h2> Part II Extract additional non-foursquare data </h2>

In [ ]:
# Load CSV Data (as geocoder happened to be to unreliable)
import wget

url = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv' # Hier dann github
wget.download(url, 'Geospatial_Coordinates.csv')

print ('File downloaded and saved!')

<h1> Part II Matching Postal-Codes to Location Data </h1>

In [ ]:
# Edit Coordinate-Table to allow mergin
Income_Table = pd.read_csv('C:/Users/klapp/Desktop/Python_Data Science/Finale/2016_Einkommen_Top15.csv',sep=';',encoding='latin1')
Income_Table.head(5)

In [ ]:
# Edit Coordinate-Table to allow mergin
Rental_Prices = pd.read_csv('C:/Users/klapp/Desktop/Python_Data Science/Finale/2019_Mietpreise.csv',sep=';',decimal=",",encoding='latin1')
Rental_Prices.head(5)

In [ ]:
# Loop through DataFrame to retrieve Location-Data
geolocator = Nominatim(user_agent="ger_explorer")

# Problem, wenn Location nicht gefunden wird, wie z.B. bei 4 (DT; Harbourfront)

Locations = []

for city in df_cities['Stadt']:
        geoinfo = geolocator.geocode(city)

        Locations.append(
               {
            'Stadt': city,
            'lat': geoinfo.latitude,
            'long':  geoinfo.longitude,
               }  
           )

         
    


Locations = pd.DataFrame(Locations)

In [165]:
df = pd.merge(df,Locations, on='Stadt')


In [ ]:
CLIENT_ID = 'TKZUDAD3BGL5SQUTQJ31AF1RO5FNLE1XB0QP23EKN3H0WLIR' # your Foursquare ID
CLIENT_SECRET = 'GGHSHKUOEXMK2E4HHLVZW0THPQODRZKS3LXBU3DMRIJ5HDVP' # your Foursquare Secret
VERSION = '20200518' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
radius = 10000 # Define radius of interest
LIMIT = 100 # How many venues do you want to check?
#catID = '4bf58dd8d48988d11f941735' 

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, catID, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            catID)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Clubs = getNearbyVenues(names=df['Stadt'],
                        latitudes=df['lat'],
                        longitudes=df['long'],
                        catID='4bf58dd8d48988d11f941735',
                        radius=5000)



In [ ]:
Clubs_grouped = Clubs.groupby('Neighbourhood').count().reset_index()
df_clubs = Clubs_grouped[['Neighbourhood','Venue']]

In [ ]:
Parks = getNearbyVenues(names=df['Stadt'],
                        latitudes=df['lat'],
                        longitudes=df['long'],
                        catID='4bf58dd8d48988d163941735',
                        radius=5000)

In [ ]:
Parks_grouped = Parks.groupby('Neighbourhood').count().reset_index()
df_parks = Parks_grouped[['Neighbourhood','Venue']]

In [ ]:
df_clubs.columns = ['Stadt','Anzahl Clubs']
df_parks.columns = ['Stadt','Anzahl Parks']

In [ ]:
Restaurants = getNearbyVenues(names=df_cities['Stadt'],
                        latitudes=df_cities['lat'],
                        longitudes=df_cities['long'],
                        catID='4d4b7105d754a06374d81259',
                        radius=5000)

In [150]:
# one hot encoding
cityfood_onehot = pd.get_dummies(Restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cityfood_onehot['Neighbourhood'] = Restaurants['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [cityfood_onehot.columns[-1]] + list(cityfood_onehot.columns[:-1])
cityfood_onehot = cityfood_onehot[fixed_columns]

cityfood_grouped = cityfood_onehot.groupby('Neighbourhood').mean().reset_index()

cityfood_grouped.head()
cityfood_grouped=cityfood_grouped.rename({'Neighbourhood':'Stadt'}, axis=1)

In [162]:
Nightlife =  getNearbyVenues(names=df_cities['Stadt'],
                        latitudes=df_cities['lat'],
                        longitudes=df_cities['long'],
                        catID='4d4b7105d754a06376d81259',
                        radius=5000)

Berlin
München
Nürnberg
Hannover
Stuttgart
Hamburg
Leipzig
Düsseldorf
Dresden
Köln
Dortmund
Bremen
Essen


In [187]:
# one hot encoding
nightlife_onehot = pd.get_dummies(Nightlife[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nightlife_onehot['Neighbourhood'] = Nightlife['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [nightlife_onehot.columns[-1]] + list(nightlife_onehot.columns[:-1])
nightlife_onehot = nightlife_onehot[fixed_columns]

nightlife_grouped = nightlife_onehot.groupby('Neighbourhood').mean().reset_index()
nightlife_grouped=nightlife_grouped.rename({'Neighbourhood':'Stadt'}, axis=1)
nightlife_grouped.head()


,Stadt,African Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Bistro,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cuban Restaurant,Deli / Bodega,Diner,Dive Bar,Eastern European Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Gastropub,Gay Bar,General Entertainment,German Restaurant,Greek Restaurant,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Jazz Club,Karaoke Bar,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Nightclub,Other Nightlife,Piano Bar,Pizza Place,Polish Restaurant,Pub,Restaurant,Rhenisch Restaurant,Rock Club,Roof Deck,Seafood Restaurant,Smoke Shop,Spanish Restaurant,Speakeasy,Sports Bar,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery
0,Berlin,0.0,0.000000,0.00,0.000000,0.0,0.280000,0.0,0.0,0.02,0.050000,0.040000,0.0,0.010000,0.030000,0.000000,0.000000,0.010000,0.150000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.010000,0.010000,0.000000,0.0,0.0,0.0,0.000000,0.040000,0.02,0.0,0.000000,0.000000,0.010000,0.0,0.020000,0.0,0.000000,0.01,0.01,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.01,0.0,0.0,0.030000,0.000000,0.00,0.0,0.0,0.00,0.00,0.0,0.010000,0.010000,0.0,0.0,0.000000,0.0,0.130000,0.0,0.000000
1,Bremen,0.0,0.000000,0.00,0.000000,0.0,0.288136,0.0,0.0,0.00,0.016949,0.033898,0.0,0.000000,0.050847,0.016949,0.000000,0.033898,0.084746,0.00,0.000000,0.016949,0.0,0.0,0.000000,0.000000,0.000000,0.016949,0.0,0.0,0.0,0.016949,0.050847,0.00,0.0,0.033898,0.000000,0.000000,0.0,0.000000,0.0,0.016949,0.00,0.00,0.050847,0.000000,0.00,0.067797,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.084746,0.000000,0.00,0.0,0.0,0.101695,0.000000,0.00,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.016949,0.0,0.000000
2,Dortmund,0.0,0.014706,0.00,0.014706,0.0,0.220588,0.0,0.0,0.00,0.014706,0.014706,0.0,0.000000,0.044118,0.000000,0.014706,0.014706,0.000000,0.00,0.014706,0.000000,0.0,0.0,0.014706,0.014706,0.000000,0.014706,0.0,0.0,0.0,0.000000,0.058824,0.00,0.0,0.000000,0.014706,0.014706,0.0,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.014706,0.00,0.029412,0.029412,0.014706,0.00,0.000000,0.014706,0.000000,0.176471,0.014706,0.00,0.0,0.0,0.102941,0.044118,0.00,0.0,0.0,0.00,0.00,0.0,0.000000,0.044118,0.0,0.0,0.014706,0.0,0.000000,0.0,0.000000
3,Dresden,0.0,0.000000,0.00,0.000000,0.0,0.175676,0.0,0.0,0.00,0.013514,0.067568,0.0,0.013514,0.040541,0.000000,0.000000,0.013514,0.067568,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.013514,0.000000,0.0,0.0,0.0,0.000000,0.040541,0.00,0.0,0.081081,0.000000,0.013514,0.0,0.027027,0.0,0.027027,0.00,0.00,0.013514,0.013514,0.00,0.040541,0.000000,0.000000,0.00,0.027027,0.000000,0.013514,0.121622,0.013514,0.00,0.0,0.0,0.067568,0.027027,0.00,0.0,0.0,0.00,0.00,0.0,0.013514,0.000000,0.0,0.0,0.013514,0.0,0.027027,0.0,0.013514
4,Düsseldorf,0.0,0.000000,0.01,0.000000,0.0,0.210000,0.0,0.0,0.01,0.010000,0.040000,0.0,0.000000,0.110000,0.000000,0.000000,0.020000,0.090000,0.01,0.000000,0.010000,0.0,0.0,0.000000,0.010000,0.000000,0.000000,0.0,0.0,0.0,0.040000,0.090000,0.00,0.0,0.040000,0.000000,0.010000,0.0,0.020000,0.0,0.000000,0.00,0.00,0.000000,0.010000,0.00,0.010000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.0,0.0,0.090000,0.050000,0.01,0.0,0.0,0.01,0.01,0.0,0.000000,0.010000,0.0,0.0,0.000000,0.0,0.020000,0.0,0.000000


In [188]:
# Merge Data 
Price_Income = pd.merge(Rental_Prices, Income_Table, on='Stadt')
df_cities = pd.merge(Price_Income,df, on='Stadt')
df_cities.head(5)

,Stadt,Mietpreis H1 2019 in Euro/m²,Verfügbares Einkommen in Euro,Size,lat,long
0,Berlin,11.6,19.719,3644826,52.517037,13.388860
1,München,18.6,29.685,1471508,48.137108,11.575382
2,Nürnberg,10.0,21.785,518365,49.453872,11.077298
3,Hannover,8.8,20.977,538068,52.374478,9.738553
4,Stuttgart,13.0,25.012,634830,48.778449,9.180013


In [168]:
# Merge Data 
df_cities = pd.merge(df_cities,df_clubs, on='Stadt')
df_cities = pd.merge(df_cities,df_parks, on='Stadt')

In [169]:
df_cities = pd.merge(df_cities,cityfood_grouped, on="Stadt")

In [189]:
df_cities = pd.merge(df_cities,nightlife_grouped, on="Stadt")

In [190]:
df_cities.head()

,Stadt,Mietpreis H1 2019 in Euro/m²,Verfügbares Einkommen in Euro,Size,lat,long,African Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Bistro,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cuban Restaurant,Deli / Bodega,Diner,Dive Bar,Eastern European Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Gastropub,Gay Bar,General Entertainment,German Restaurant,Greek Restaurant,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Jazz Club,Karaoke Bar,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Nightclub,Other Nightlife,Piano Bar,Pizza Place,Polish Restaurant,Pub,Restaurant,Rhenisch Restaurant,Rock Club,Roof Deck,Seafood Restaurant,Smoke Shop,Spanish Restaurant,Speakeasy,Sports Bar,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery
0,Berlin,11.6,19.719,3644826,52.517037,13.388860,0.000000,0.0,0.0,0.0,0.0,0.280000,0.00,0.000000,0.020000,0.050000,0.040000,0.000000,0.01,0.030000,0.000000,0.0,0.010000,0.150000,0.000000,0.0,0.0,0.0,0.00,0.0,0.01,0.01,0.000000,0.00,0.00,0.00,0.000000,0.040000,0.020000,0.0,0.000000,0.000000,0.010000,0.000000,0.020000,0.0,0.0,0.010000,0.01,0.010000,0.000000,0.01,0.020000,0.0,0.0,0.0,0.000000,0.0,0.00,0.050000,0.0,0.01,0.0,0.000000,0.030000,0.000000,0.0,0.000000,0.00,0.0,0.0,0.00,0.010000,0.010000,0.000000,0.0,0.0,0.000000,0.130000,0.00,0.0
1,München,18.6,29.685,1471508,48.137108,11.575382,0.000000,0.0,0.0,0.0,0.0,0.250000,0.05,0.000000,0.000000,0.020000,0.070000,0.000000,0.00,0.020000,0.000000,0.0,0.070000,0.160000,0.010000,0.0,0.0,0.0,0.01,0.0,0.00,0.00,0.000000,0.01,0.01,0.01,0.000000,0.030000,0.010000,0.0,0.050000,0.000000,0.000000,0.000000,0.030000,0.0,0.0,0.000000,0.00,0.030000,0.000000,0.00,0.010000,0.0,0.0,0.0,0.000000,0.0,0.01,0.020000,0.0,0.00,0.0,0.000000,0.040000,0.000000,0.0,0.000000,0.01,0.0,0.0,0.01,0.010000,0.010000,0.000000,0.0,0.0,0.000000,0.030000,0.01,0.0
2,Nürnberg,10.0,21.785,518365,49.453872,11.077298,0.000000,0.0,0.0,0.0,0.0,0.253968,0.00,0.015873,0.000000,0.015873,0.079365,0.000000,0.00,0.031746,0.000000,0.0,0.079365,0.079365,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.063492,0.000000,0.0,0.015873,0.015873,0.047619,0.000000,0.047619,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.031746,0.0,0.0,0.0,0.015873,0.0,0.00,0.063492,0.0,0.00,0.0,0.015873,0.047619,0.031746,0.0,0.000000,0.00,0.0,0.0,0.00,0.015873,0.000000,0.000000,0.0,0.0,0.000000,0.031746,0.00,0.0
3,Hannover,8.8,20.977,538068,52.374478,9.738553,0.000000,0.0,0.0,0.0,0.0,0.118421,0.00,0.000000,0.026316,0.013158,0.118421,0.013158,0.00,0.039474,0.013158,0.0,0.026316,0.105263,0.013158,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.052632,0.000000,0.0,0.013158,0.000000,0.039474,0.000000,0.013158,0.0,0.0,0.013158,0.00,0.013158,0.000000,0.00,0.039474,0.0,0.0,0.0,0.000000,0.0,0.00,0.092105,0.0,0.00,0.0,0.000000,0.144737,0.026316,0.0,0.000000,0.00,0.0,0.0,0.00,0.000000,0.013158,0.013158,0.0,0.0,0.000000,0.039474,0.00,0.0
4,Stuttgart,13.0,25.012,634830,48.778449,9.180013,0.011111,0.0,0.0,0.0,0.0,0.244444,0.00,0.000000,0.011111,0.000000,0.077778,0.000000,0.00,0.011111,0.000000,0.0,0.044444,0.077778,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.011111,0.00,0.00,0.00,0.011111,0.033333,0.011111,0.0,0.055556,0.000000,0.022222,0.011111,0.022222,0.0,0.0,0.000000,0.00,0.000000,0.011111,0.00,0.055556,0.0,0.0,0.0,0.000000,0.0,0.00,0.100000,0.0,0.00,0.0,0.000000,0.044444,0.022222,0.0,0.011111,0.00,0.0,0.0,0.00,0.000000,0.033333,0.000000,0.0,0.0,0.011111,0.055556,0.00,0.0


In [204]:
# set number of clusters
kclusters = 5

df_clustered = df_cities.drop('Stadt', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustered.drop(['Size','lat','long'],1))


In [205]:
# add clustering labels
#df_cities.insert(0, 'Cluster Labels', kmeans.labels_)
df_cities['Cluster Labels'] = kmeans.labels_

#ger_merged = df_cities

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#ger_merged = ger_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [182]:
df_cities

,Stadt,Mietpreis H1 2019 in Euro/m²,Verfügbares Einkommen in Euro,Size,lat,long,African Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Bistro,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cuban Restaurant,Deli / Bodega,Diner,Dive Bar,Eastern European Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Gastropub,Gay Bar,General Entertainment,German Restaurant,Greek Restaurant,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Jazz Club,Karaoke Bar,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Nightclub,Other Nightlife,Piano Bar,Pizza Place,Polish Restaurant,Pub,Restaurant,Rhenisch Restaurant,Rock Club,Roof Deck,Seafood Restaurant,Smoke Shop,Spanish Restaurant,Speakeasy,Sports Bar,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Cluster Labels
0,Berlin,11.6,19.719,3644826,52.517037,13.388860,0.000000,0.000000,0.00,0.000000,0.000000,0.280000,0.00,0.000000,0.020000,0.050000,0.040000,0.000000,0.010000,0.030000,0.000000,0.000000,0.010000,0.150000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.00,0.00,0.00,0.000000,0.040000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.020000,0.000000,0.000000,0.010000,0.01,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.01,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.130000,0.00,0.000000,0
1,München,18.6,29.685,1471508,48.137108,11.575382,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.05,0.000000,0.000000,0.020000,0.070000,0.000000,0.000000,0.020000,0.000000,0.000000,0.070000,0.160000,0.010000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.01,0.01,0.01,0.000000,0.030000,0.010000,0.000000,0.050000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.00,0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.010000,0.020000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.01,0.00,0.00,0.01,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.030000,0.01,0.000000,1
2,Nürnberg,10.0,21.785,518365,49.453872,11.077298,0.000000,0.000000,0.00,0.000000,0.000000,0.253968,0.00,0.015873,0.000000,0.015873,0.079365,0.000000,0.000000,0.031746,0.000000,0.000000,0.079365,0.079365,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.063492,0.000000,0.000000,0.015873,0.015873,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.031746,0.000000,0.000000,0.00,0.015873,0.000000,0.000000,0.063492,0.000000,0.00,0.000000,0.015873,0.047619,0.031746,0.000000,0.000000,0.00,0.00,0.00,0.00,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.031746,0.00,0.000000,0
3,Hannover,8.8,20.977,538068,52.374478,9.738553,0.000000,0.000000,0.00,0.000000,0.000000,0.118421,0.00,0.000000,0.026316,0.013158,0.118421,0.013158,0.000000,0.039474,0.013158,0.000000,0.026316,0.105263,0.013158,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.052632,0.000000,0.000000,0.013158,0.000000,0.039474,0.000000,0.013158,0.000000,0.000000,0.013158,0.00,0.013158,0.000000,0.000000,0.039474,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.092105,0.000000,0.00,0.000000,0.000000,0.144737,0.026316,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.013158,0.013158,0.000000,0.000000,0.000000,0.039474,0.00,0.000000,0
4,Stuttgart,13.0,25.012,634830,48.778449,9.180013,0.011111,0.000000,0.00,0.000000,0.000000,0.244444,0.00,0.000000,0.011111,0.000000,0.077778,0.000000,0.000000,0.011111,0.000000,0.000000,0

In [199]:
df_cities.loc[df_cities['Cluster Labels'] == 0]

,Stadt,Mietpreis H1 2019 in Euro/m²,Verfügbares Einkommen in Euro,Size,lat,long,African Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Bistro,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cuban Restaurant,Deli / Bodega,Diner,Dive Bar,Eastern European Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Gastropub,Gay Bar,General Entertainment,German Restaurant,Greek Restaurant,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Jazz Club,Karaoke Bar,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Nightclub,Other Nightlife,Piano Bar,Pizza Place,Polish Restaurant,Pub,Restaurant,Rhenisch Restaurant,Rock Club,Roof Deck,Seafood Restaurant,Smoke Shop,Spanish Restaurant,Speakeasy,Sports Bar,Theater,Theme Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery,Cluster Labels
0,Berlin,11.6,19.719,3644826,52.517037,13.388860,0.0,0.0,0.0,0.0,0.0,0.280000,0.0,0.000000,0.020000,0.050000,0.040000,0.000000,0.01,0.030000,0.000000,0.0,0.010000,0.150000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.01,0.01,0.000000,0.0,0.0,0.0,0.000000,0.040000,0.02,0.0,0.000000,0.000000,0.010000,0.0,0.020000,0.0,0.000000,0.010000,0.01,0.010000,0.0,0.01,0.020000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.050000,0.0,0.01,0.000000,0.000000,0.030000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.010000,0.010000,0.000000,0.0,0.000000,0.0,0.130000,0.0,0.0,0
2,Nürnberg,10.0,21.785,518365,49.453872,11.077298,0.0,0.0,0.0,0.0,0.0,0.253968,0.0,0.015873,0.000000,0.015873,0.079365,0.000000,0.00,0.031746,0.000000,0.0,0.079365,0.079365,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.063492,0.00,0.0,0.015873,0.015873,0.047619,0.0,0.047619,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.031746,0.000000,0.000000,0.0,0.015873,0.0,0.0,0.063492,0.0,0.00,0.000000,0.015873,0.047619,0.031746,0.000000,0.0,0.0,0.0,0.0,0.0,0.015873,0.000000,0.000000,0.0,0.000000,0.0,0.031746,0.0,0.0,0
3,Hannover,8.8,20.977,538068,52.374478,9.738553,0.0,0.0,0.0,0.0,0.0,0.118421,0.0,0.000000,0.026316,0.013158,0.118421,0.013158,0.00,0.039474,0.013158,0.0,0.026316,0.105263,0.013158,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.052632,0.00,0.0,0.013158,0.000000,0.039474,0.0,0.013158,0.0,0.000000,0.013158,0.00,0.013158,0.0,0.00,0.039474,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.092105,0.0,0.00,0.000000,0.000000,0.144737,0.026316,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.013158,0.013158,0.0,0.000000,0.0,0.039474,0.0,0.0,0
9,Köln,10.9,21.608,1085664,50.938361,6.959974,0.0,0.0,0.0,0.0,0.0,0.180556,0.0,0.000000,0.000000,0.000000,0.069444,0.000000,0.00,0.125000,0.000000,0.0,0.013889,0.138889,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.055556,0.00,0.0,0.055556,0.000000,0.013889,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.055556,0.013889,0.013889,0.0,0.000000,0.0,0.0,0.069444,0.0,0.00,0.013889,0.000000,0.069444,0.041667,0.013889,0.0,0.0,0.0,0.0,0.0,0.000000,0.027778,0.000000,0.0,0.013889,0.0,0.013889,0.0,0.0,0
11,Bremen,8.2,21.327,569352,53.075820,8.807165,0.0,0.0,0.0,0.0,0.0,0.288136,0.0,0.000000,0.000000,0.016949,0.033898,0.000000,0.00,0.050847,0.016949,0.0,0.033898,0.084746,0.000000,0.0,0.016949,0.0,0.0,0.0,0.00,0.00,0.016949,0.0,0.0,0.0,0.016949,0.050847,0.00,0.0,0.033898,0.000000,0.000000,0.0,0.000000,0.0,0.016949,0.000000,0.00,0.050847,0.0,0.00,0.067797,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.084746,0.0,0.00,0.000000,0.000000,0.101695,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.016949,0.0,0.0,0


In [220]:
# create a map, centered around the mean (i.e. Median) toronto Latitude and Longitude Values

map_clusters = folium.Map(location=[49.8, 12], zoom_start=6)

# set color scheme for the clusters / note I changed it compared to the Manhattan - Task; as I added a "missing value category i.e. No Venues in 500m radius"
x = np.arange(df_cities['Cluster Labels'].nunique())
ys = [i + x + (i*x)**2 for i in range(df_cities['Cluster Labels'].nunique())]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_cities['lat'], df_cities['long'], df_cities['Stadt'], df_cities['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [200]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [207]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Stadt']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Stadt'] = nightlife_grouped['Stadt']

for ind in np.arange(nightlife_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nightlife_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Stadt,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berlin,Bar,Cocktail Bar,Wine Bar,Beer Bar,Nightclub,Gastropub,Beer Garden,Pub,Brewery,Gay Bar
1,Bremen,Bar,Pub,Cocktail Bar,Nightclub,Lounge,Gastropub,Italian Restaurant,Brewery,Beer Garden,Café
2,Dortmund,Bar,Nightclub,Pub,Gastropub,Sports Bar,Restaurant,Brewery,Lounge,Mediterranean Restaurant,Hookah Bar
3,Dresden,Bar,Nightclub,German Restaurant,Cocktail Bar,Beer Garden,Pub,Lounge,Gastropub,Brewery,Indie Movie Theater
4,Düsseldorf,Bar,Brewery,Gastropub,Cocktail Bar,Pub,Restaurant,German Restaurant,Nightclub,Beer Garden,French Restaurant
5,Essen,Nightclub,Bar,Cocktail Bar,Pub,Café,Restaurant,Gastropub,Greek Restaurant,Burger Joint,Brewery
6,Hamburg,Cocktail Bar,Bar,Pub,Nightclub,Wine Bar,Gastropub,Café,German Restaurant,Brewery,Beach Bar
7,Hannover,Pub,Bar,Beer Garden,Cocktail Bar,Nightclub,Gastropub,Wine Bar,Lounge,Brewery,Hookah Bar
8,Köln,Bar,Cocktail Bar,Brewery,Nightclub,Beer Garden,Pub,German Restaurant,Lounge,Gastropub,Restaurant
9,Leipzig,Bar,Nightclub,Pub,Gastropub,Café,Beer Garden,Cocktail Bar,German Restaurant,Hookah Bar,Lounge
